In [1]:
import pandas as pd
import numpy as np
import sys
import datetime

<B>Data Exploration</B>

In [2]:
meter_list_df = pd.read_excel('gorilla_test_data.xlsx', sheet_name='meter_list')
forecast_table_df = pd.read_excel('gorilla_test_data.xlsx', sheet_name='forecast_table')
rate_table_df = pd.read_excel('gorilla_test_data.xlsx', sheet_name='rate_table')


In [4]:
meter_list_df

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


In [6]:
forecast_table_df

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134
...,...,...,...
3407,88357331,2022-09-27,441.014725
3408,88357331,2022-09-28,441.512055
3409,88357331,2022-09-29,437.240657
3410,88357331,2022-09-30,517.608354


In [3]:
rate_table_df = rate_table_df.replace(np.nan, int(sys.maxsize))

<B>Part 1 </B>

In [109]:
t_agg = pd.merge(meter_list_df,forecast_table_df, how='inner', on='meter_id')
t_agg.loc[:,'rate_pound']=None

In [110]:
t_agg

,meter_id,aq_kwh,exit_zone,date,kwh,rate_pound
0,14676236,28978,EA1,2020-06-01,22.070768,None
1,14676236,28978,EA1,2020-06-02,19.170720,None
2,14676236,28978,EA1,2020-06-03,23.555111,None
3,14676236,28978,EA1,2020-06-04,18.220712,None
4,14676236,28978,EA1,2020-06-05,14.196134,None
...,...,...,...,...,...,...
3407,88357331,484399,SE2,2022-09-27,441.014725,None
3408,88357331,484399,SE2,2022-09-28,441.512055,None
3409,88357331,484399,SE2,2022-09-29,437.240657,None
3410,88357331,484399,SE2,2022-09-30,517.608354,None


In [111]:
def cost_extraction(r):
    costs =  rate_table_df[(rate_table_df['exit_zone']==r.exit_zone) & (rate_table_df['aq_min_kwh']<=r.aq_kwh) & (rate_table_df['aq_max_kwh']>r.aq_kwh) & (rate_table_df['date']<=r.date)]
    r.rate_pound= costs['rate_p_per_kwh'].iloc[-1] * 0.01
    return r

In [112]:
t_agg = t_agg.apply(cost_extraction, axis=1)

In [113]:
t_agg.iloc[669]

meter_id                 14676236
aq_kwh                      28978
exit_zone                     EA1
date          2022-04-01 00:00:00
kwh                     47.160597
rate_pound               0.004173
Name: 669, dtype: object

In [114]:
def calculate_vals(r):
    condition = t_agg['meter_id']==r.meter_id
    r.total_consumption_kwh = t_agg.loc[condition, 'kwh'].sum()
    r.total_cost_pound = t_agg.loc[condition,'kwh'].multiply(t_agg.loc[condition,'rate_pound']).sum()
    return r


In [208]:
aggregate_df = pd.DataFrame({'meter_id':meter_list_df.meter_id, 'total_consumption_kwh': None , 'total_cost_pound': None }).apply(calculate_vals, axis=1)
aggregate_df

,meter_id,total_consumption_kwh,total_cost_pound
0,14676236,28978.0,100.152012
1,34509937,78324.0,275.489119
2,50264822,265667.0,731.244071
3,88357331,484399.0,1433.160171


<B>Part 2</B>

Generate List of Meters of Any Size

In [4]:
def generate_meter_list(list_size):
    return pd.DataFrame({'meter_id': np.random.randint(10000000, 100000000, size=list_size), 'aq_kwh':np.random.randint(0, 10000000, size=list_size), 'exit_zone': np.random.choice(rate_table_df['exit_zone'].unique(), size=list_size)})

In [131]:
meter_generated = generate_meter_list(100)

<B>Part 3</B>

Generate Mock Consumption Data:

In [5]:
def generate_consumption_data(random_meter_list, start_date, duration_days):

    meter_ids = np.repeat(random_meter_list['meter_id'].unique(), duration_days)
    # dates = [start_date + datetime.timedelta(days=x) for x in range(duration_days)]
    dates = pd.date_range(start=start_date, periods = duration_days)
    dates_full_length = np.tile(dates,random_meter_list['meter_id'].unique().shape[0])
    
    kwh_vals = np.random.rand(len(dates_full_length))*50

    return pd.DataFrame({'meter_id': meter_ids, 'date': dates_full_length, 'kwh':kwh_vals})



In [223]:
consumption_generated = generate_consumption_data(meter_generated, datetime.date(2020,5,1), 5)

In [224]:
consumption_generated.head(10)

,meter_id,date,kwh
0,12790395,2020-05-01,7.657974
1,12790395,2020-05-02,11.798057
2,12790395,2020-05-03,35.180718
3,12790395,2020-05-04,17.981338
4,12790395,2020-05-05,35.162448
5,44048148,2020-05-01,8.814744
6,44048148,2020-05-02,34.355347
7,44048148,2020-05-03,3.425428
8,44048148,2020-05-04,30.270272
9,44048148,2020-05-05,37.190535


In [225]:
consumption_generated['date']

0     2020-05-01
1     2020-05-02
2     2020-05-03
3     2020-05-04
4     2020-05-05
         ...    
495   2020-05-01
496   2020-05-02
497   2020-05-03
498   2020-05-04
499   2020-05-05
Name: date, Length: 500, dtype: datetime64[ns]

<B>Part 4</B>

Function to compute consumption data

In [6]:
def compute_consumption_data(meter_list, consumption_forecast):
    def cost_extraction(r):
        costs =  rate_table_df[(rate_table_df['exit_zone']==r.exit_zone) & (rate_table_df['aq_min_kwh']<=r.aq_kwh) & (rate_table_df['aq_max_kwh']>r.aq_kwh) & (rate_table_df['date']<=r.date)]
        r.rate_pound= costs['rate_p_per_kwh'].iloc[-1] * 0.01
        return r
    
    def calculate_vals(r):
        condition = t_agg['meter_id']==r.meter_id
        r.total_consumption_kwh = t_agg.loc[condition, 'kwh'].sum()
        r.total_cost_pound = t_agg.loc[condition,'kwh'].multiply(t_agg.loc[condition,'rate_pound']).sum()
        return r
        
    t_agg = pd.merge(meter_list,consumption_forecast, how='inner', on='meter_id')
    t_agg.loc[:,'rate_pound']=None
    t_agg = t_agg.apply(cost_extraction, axis=1)

    aggregate_df = pd.DataFrame({'meter_id':meter_list.meter_id, 'total_consumption_kwh': None , 'total_cost_pound': None }).apply(calculate_vals, axis=1)

    return aggregate_df


In [251]:
compute_consumption_data(meter_generated, consumption_generated)

,meter_id,total_consumption_kwh,total_cost_pound
0,12790395,107.780534,0.330240
1,44048148,114.056327,0.265751
2,25795088,171.543308,0.331593
3,37354305,108.731108,0.251060
4,26334458,137.602629,0.410606
...,...,...,...
95,74998320,106.659015,0.274220
96,17458597,81.673024,0.192912
97,91427460,143.045487,0.337873
98,74285771,169.117017,0.488579


<B>Part 4: Benchmarking</B>

In [12]:
# Benchmark 
import time
meter_sizes = [100, 500]
durations = [50, 100, 500]
times = []
for meter_size in meter_sizes:
    for duration in durations:
        print('generating meters list ..')
        meter_generated_benchmark = generate_meter_list(meter_size)
        print('generating forecasts ..')
        forecast_generated_benchmark = generate_consumption_data(meter_generated_benchmark, datetime.date(2020, 4,1), duration)
        print('computing consumption data .. ')
        tic = time.time()
        computed_benchmark = compute_consumption_data(meter_generated_benchmark, forecast_generated_benchmark)
        toc = time.time()
        print( f'meter size: {meter_size} & duration: {duration}, time= {toc-tic}s')
        times.append(toc-tic)

generating meters list ..
generating forecasts ..
computing consumption data .. 
meter size: 100 & duration: 50, time= 16.470703601837158s
generating meters list ..
generating forecasts ..
computing consumption data .. 
meter size: 100 & duration: 100, time= 32.16406035423279s
generating meters list ..
generating forecasts ..
computing consumption data .. 
meter size: 100 & duration: 500, time= 120.98808288574219s
generating meters list ..
generating forecasts ..
computing consumption data .. 
meter size: 500 & duration: 50, time= 54.61790657043457s
generating meters list ..
generating forecasts ..
computing consumption data .. 
meter size: 500 & duration: 100, time= 121.87677788734436s
generating meters list ..
generating forecasts ..
computing consumption data .. 
meter size: 500 & duration: 500, time= 780.902108669281s


<B>Observations:</B>

We notice from the results that the function doesn't scale very well with a growing size of data. One possible improvement can be to further transform the data to parallelize and vectorize the processes better. 